## Ejemplo 3: Clasificación Semántica

Charla: Potenciando la Inteligencia Artificial con ROS y OpenVINO

Autor: Dr. Edgar Macías García 

Human Robot Collaboration Group

Intel Labs

### 1.- Importar librerias

In [ ]:
#Instalar dependencias
!pip3 install openvino

In [ ]:
#Librerias basicas
import numpy as np
import matplotlib.pyplot as plt
import os

#Herramientas de manejo de imagenes
from PIL import Image

#Herramientas de manejo de directorios
import glob

#Importar OpenVINO
from openvino.inference_engine import IENetwork
from openvino.inference_engine import IECore

### 2.- Importar repositorio

In [ ]:
#Obtener repositorio de github
!git clone https://github.com/iqedgarmg/itesm_workshop 

#Establecer directorio principal
os.chdir('/content/itesm_workshop/ejemplos/semantica')

### 3.- Definir funcionas auxiliares

a) Preprocesamiento

In [ ]:
#Preprocesar imagen
def prep_image(img, mean, std, size):

  #Redimensionar imagen
  image = img.resize((size[0], size[1]))
  img = np.array(image)
    
  #Reordenar a dimensiones de la red
  img = img[np.newaxis, :, :, :].astype(np.float32)

  #Preprocesamiento: Quitar media
  img[0,:,:,0] -= mean[0]
  img[0,:,:,1] -= mean[1]
  img[0,:,:,2] -= mean[2]

  #Preprocesamiento: Normalizar
  img[0,:,:,0] /= std[0]
  img[0,:,:,1] /= std[1]
  img[0,:,:,2] /= std[2]

  return img  

b) Inferencia

In [ ]:
#Cargar modelo a IR
def load_ir_model(model_xml, model_bin):

  #Cargar plugin de IE
  plugin = IECore()

  #Cargar modelo
  net = plugin.read_network(model=model_xml, weights=model_bin)
  exec_net = plugin.load_network(network=net, device_name="CPU", num_requests=1)
  print("Modelo cargado en IE")
  return exec_net

#Syncrhonous inference
def ir_inference(executable_net, image):

  #Get input blob for the inference request
  input_blob = next(iter(executable_net.input_info))

  #Perform synchronous inference
  result = executable_net.infer(inputs = {input_blob: image})

  return result

c) Post-procesamiento

In [ ]:
#Obtener segmentacion
def get_segmentation(out_ov, size):

  #Filtrar clases
  im_out_ov = np.zeros(out_ov.shape[0]).astype("uint8")
  for j in range(out_ov.shape[0]):

    #Obtener clase de pixel
    index = np.argmax(out_ov[j,:])
    im_out_ov[j] = index
    
  #Redimensionar imagen
  im_out_ov = im_out_ov.reshape([size[0], size[1]])

  return im_out_ov

### 4.- Cargar datos

In [ ]:
#Cargar ruta de imagenes de prueba
path = glob.glob("images/*.png")

print("Datos de entrada: " + str(len(path)))

#Mostrar ejemplo
img = Image.open(path[0])
print("Dimensiones: " + str(img.size))

plt.figure(1, figsize=(5,5))
plt.imshow(img)

5.- Cargar modelo

In [ ]:
#Crear instancia del modelo


#Imprimir informacion del modelo


6.- Procesar entradas

In [ ]:
#Leer imagen


#Preprocesar entrada


#Ejecutar inferencia


#Correr post-procesamiento (Interpretar salida)


#Imprimir figura
plt.figure(1)
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.subplot(1, 2, 2)
plt.imshow(img_seg)
plt.show()